In [1]:
import anndata as ad
import gc
import sys
from scipy.sparse import csc_matrix
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression
import random
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import pickle
import matlab.engine
import heapq
import ipyvolume as ipv

In [2]:
eng = matlab.engine.start_matlab()

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

cuda


In [4]:
# par_RNA_DNA = {
#     # RNA, 16394 * 13431
#     'input_train_mod1': 'output/datasets/predict_modality/openproblems_bmmc_multiome_phase1_rna/openproblems_bmmc_multiome_phase1_rna.censor_dataset.output_train_mod1.h5ad',
#     # DNA, 16394 * 10000
#     'input_train_mod2': 'output/datasets/predict_modality/openproblems_bmmc_multiome_phase1_rna/openproblems_bmmc_multiome_phase1_rna.censor_dataset.output_train_mod2.h5ad',
#     # RNA, 1000 * 13431
#     'input_test_mod1': 'output/datasets/predict_modality/openproblems_bmmc_multiome_phase1_rna/openproblems_bmmc_multiome_phase1_rna.censor_dataset.output_test_mod1.h5ad',
#     # DNA, 1000 * 10000
#     'input_test_mod2': 'output/datasets/predict_modality/openproblems_bmmc_multiome_phase1_rna/openproblems_bmmc_multiome_phase1_rna.censor_dataset.output_test_mod2.h5ad',
#     'distance_method': 'minkowski',
#     'output': 'output.h5ad',
#     'n_pcs': 50,
# }

# par_DNA_RNA = {
#     # DNA, 16394 * 116490
#     'input_train_mod1': 'output/datasets/predict_modality/openproblems_bmmc_multiome_phase1_mod2/openproblems_bmmc_multiome_phase1_mod2.censor_dataset.output_train_mod1.h5ad',
#     # RNA, 16394 * 13431
#     'input_train_mod2': 'output/datasets/predict_modality/openproblems_bmmc_multiome_phase1_mod2/openproblems_bmmc_multiome_phase1_mod2.censor_dataset.output_train_mod2.h5ad',
#     # DNA, 1000 * 116490
#     'input_test_mod1': 'output/datasets/predict_modality/openproblems_bmmc_multiome_phase1_mod2/openproblems_bmmc_multiome_phase1_mod2.censor_dataset.output_test_mod1.h5ad',
#     # RNA, 1000 * 13431
#     'input_test_mod2': 'output/datasets/predict_modality/openproblems_bmmc_multiome_phase1_mod2/openproblems_bmmc_multiome_phase1_mod2.censor_dataset.output_test_mod2.h5ad',
#     'distance_method': 'minkowski',
#     'output': 'output.h5ad',
#     'n_pcs': 50,
# }

# par_RNA_Pro = {
#     # RNA, 29077 * 13953
#     'input_train_mod1': 'output/datasets/predict_modality/openproblems_bmmc_cite_phase1_rna/openproblems_bmmc_cite_phase1_rna.censor_dataset.output_train_mod1.h5ad',
#     # Protein, 29077 * 134
#     'input_train_mod2': 'output/datasets/predict_modality/openproblems_bmmc_cite_phase1_rna/openproblems_bmmc_cite_phase1_rna.censor_dataset.output_train_mod2.h5ad',
#     # RNA, 1000 * 13953
#     'input_test_mod1': 'output/datasets/predict_modality/openproblems_bmmc_cite_phase1_rna/openproblems_bmmc_cite_phase1_rna.censor_dataset.output_test_mod1.h5ad',
#     # Protein, 1000 * 134
#     'input_test_mod2': 'output/datasets/predict_modality/openproblems_bmmc_cite_phase1_rna/openproblems_bmmc_cite_phase1_rna.censor_dataset.output_test_mod2.h5ad',
#     'distance_method': 'minkowski',
#     'output': 'output.h5ad',
#     'n_pcs': 50,
# }

# par_Pro_RNA = {
#     # Protein, 29077 * 134
#     'input_train_mod1': 'output/datasets/predict_modality/openproblems_bmmc_cite_phase1_mod2/openproblems_bmmc_cite_phase1_mod2.censor_dataset.output_train_mod1.h5ad',
#     # RNA, 29077 * 13953
#     'input_train_mod2': 'output/datasets/predict_modality/openproblems_bmmc_cite_phase1_mod2/openproblems_bmmc_cite_phase1_mod2.censor_dataset.output_train_mod2.h5ad',
#     # Protein. 1000 * 134
#     'input_test_mod1': 'output/datasets/predict_modality/openproblems_bmmc_cite_phase1_mod2/openproblems_bmmc_cite_phase1_mod2.censor_dataset.output_test_mod1.h5ad',
#     # RNA, 1000 * 13953
#     'input_test_mod2': 'output/datasets/predict_modality/openproblems_bmmc_cite_phase1_mod2/openproblems_bmmc_cite_phase1_mod2.censor_dataset.output_test_mod2.h5ad',
#     'distance_method': 'minkowski',
#     'output': 'output.h5ad',
#     'n_pcs': 50,

# }

In [5]:
# par_list = [par_RNA_DNA, par_DNA_RNA, par_RNA_Pro, par_Pro_RNA]

In [6]:
# index = int(sys.argv[1])
# par = par_list[2]

In [7]:
# input_train_mod1 = ad.read_h5ad(par['input_train_mod1'])
# input_train_mod2 = ad.read_h5ad(par['input_train_mod2'])
# input_test_mod1 = ad.read_h5ad(par['input_test_mod1'])
# input_test_mod2 = ad.read_h5ad(par['input_test_mod2'])

In [8]:
# print(set(input_train_mod1.obs['batch']))
# print(set(input_train_mod2.obs['batch']))
# print(set(input_test_mod1.obs['batch']))
# print(set(input_test_mod2.obs['batch']))

In [9]:
# s2d4 = input_train_mod2[input_train_mod2.obs["batch"] == "s2d4", :]
# pro_s2d4 = s2d4

# s1d2 = input_test_mod2[input_test_mod2.obs["batch"] == "s1d2", :]
# pro_s1d2 = s1d2

In [10]:
# print(pro_s2d4)
# print(pro_s1d2)

In [11]:
train_mod1_file = 'phase2_data/predict_modality/openproblems_bmmc_multiome_phase2_mod2/openproblems_bmmc_multiome_phase2_mod2.censor_dataset.output_train_mod1.h5ad'
train_mod2_file = 'phase2_data/predict_modality/openproblems_bmmc_multiome_phase2_mod2/openproblems_bmmc_multiome_phase2_mod2.censor_dataset.output_train_mod2.h5ad'
test_mod1_file = 'phase2_data/predict_modality/openproblems_bmmc_multiome_phase2_rna/openproblems_bmmc_multiome_phase2_rna.censor_dataset.output_train_mod1.h5ad'
test_mod2_file = 'phase2_data/predict_modality/openproblems_bmmc_multiome_phase2_rna/openproblems_bmmc_multiome_phase2_rna.censor_dataset.output_train_mod2.h5ad'

In [12]:
input_train_mod1 = ad.read_h5ad(train_mod1_file)
input_train_mod2 = ad.read_h5ad(train_mod2_file)
input_test_mod1 = ad.read_h5ad(test_mod1_file)
input_test_mod2 = ad.read_h5ad(test_mod2_file)

In [13]:
print(input_train_mod1)
print(input_train_mod2)
print(input_test_mod1)
print(input_test_mod2)

AnnData object with n_obs × n_vars = 42492 × 116490
    obs: 'batch'
    var: 'feature_types'
    uns: 'dataset_id', 'gene_activity_var_names', 'organism'
    obsm: 'gene_activity'
    layers: 'counts'
AnnData object with n_obs × n_vars = 42492 × 13431
    obs: 'batch'
    var: 'gene_ids', 'feature_types'
    uns: 'dataset_id', 'organism'
    layers: 'counts'
AnnData object with n_obs × n_vars = 42492 × 13431
    obs: 'batch', 'size_factors'
    var: 'gene_ids', 'feature_types'
    uns: 'dataset_id', 'organism'
    layers: 'counts'
AnnData object with n_obs × n_vars = 42492 × 10000
    obs: 'batch', 'size_factors'
    var: 'feature_types'
    uns: 'dataset_id', 'gene_activity_var_names', 'organism'
    obsm: 'gene_activity'
    layers: 'counts'


In [14]:
print(set(input_train_mod1.obs['batch']))
print(set(input_train_mod2.obs['batch']))
print(set(input_test_mod1.obs['batch']))
print(set(input_test_mod2.obs['batch']))

{'s2d5', 's2d4', 's3d6', 's3d7', 's1d3', 's2d1', 's1d2', 's1d1', 's3d3', 's3d10'}
{'s2d5', 's2d4', 's3d6', 's3d7', 's1d3', 's2d1', 's1d2', 's1d1', 's3d3', 's3d10'}
{'s2d5', 's2d4', 's3d6', 's3d7', 's1d3', 's2d1', 's1d2', 's1d1', 's3d3', 's3d10'}
{'s2d5', 's2d4', 's3d6', 's3d7', 's1d3', 's2d1', 's1d2', 's1d1', 's3d3', 's3d10'}


In [15]:
s3d6 = input_train_mod1[input_train_mod1.obs["batch"] == "s3d6", :]
pro_s3d6 = s3d6

s2d4 = input_train_mod1[input_train_mod1.obs["batch"] == "s2d4", :]
pro_s2d4 = s2d4

s1d1 = input_train_mod1[input_train_mod1.obs["batch"] == "s1d1", :]
pro_s1d1 = s1d1

s2d1 = input_train_mod1[input_train_mod1.obs["batch"] == "s2d1", :]
pro_s2d1 = s2d1

s1d2 = input_test_mod1[input_test_mod1.obs["batch"] == "s1d2", :]
pro_s1d2 = s1d2

In [16]:
print(pro_s3d6)
print(pro_s2d4)
print(pro_s1d1)
print(pro_s2d1)
print(pro_s1d2)

View of AnnData object with n_obs × n_vars = 1771 × 116490
    obs: 'batch'
    var: 'feature_types'
    uns: 'dataset_id', 'gene_activity_var_names', 'organism'
    obsm: 'gene_activity'
    layers: 'counts'
View of AnnData object with n_obs × n_vars = 5456 × 116490
    obs: 'batch'
    var: 'feature_types'
    uns: 'dataset_id', 'gene_activity_var_names', 'organism'
    obsm: 'gene_activity'
    layers: 'counts'
View of AnnData object with n_obs × n_vars = 5616 × 116490
    obs: 'batch'
    var: 'feature_types'
    uns: 'dataset_id', 'gene_activity_var_names', 'organism'
    obsm: 'gene_activity'
    layers: 'counts'
View of AnnData object with n_obs × n_vars = 3811 × 116490
    obs: 'batch'
    var: 'feature_types'
    uns: 'dataset_id', 'gene_activity_var_names', 'organism'
    obsm: 'gene_activity'
    layers: 'counts'
View of AnnData object with n_obs × n_vars = 6069 × 13431
    obs: 'batch', 'size_factors'
    var: 'gene_ids', 'feature_types'
    uns: 'dataset_id', 'organism'
  

In [17]:
mod1 = input_train_mod1.var['feature_types'][0]
mod2 = input_train_mod2.var['feature_types'][0]

model_path = mod1 +'_to_' + mod2 + '.pth'

In [18]:
s1d1_array = pro_s1d1.X.toarray()
s1d2_array = pro_s1d2.X.toarray()
s2d1_array = pro_s2d1.X.toarray()
s2d4_array = pro_s2d4.X.toarray()
s3d6_array = pro_s3d6.X.toarray()

In [19]:
# from mayavi.mlab import *
# def test_points3d():
#     t = np.linspace(0, 4 * np.pi, 20)

#     x = np.sin(2 * t)
#     y = np.cos(t)
#     z = np.cos(2 * t)
#     s = 2 + np.sin(t)

#     return points3d(x, y, z, s, colormap="copper", scale_factor=.25)
# test_points3d()

In [20]:
# from mpl_toolkits.mplot3d import Axes3D
# import matplotlib.pyplot as plt
# import numpy as np
# from itertools import product, combinations

In [21]:
# fig = plt.figure()
# ax = fig.gca(projection='3d')
# ax.set_aspect("auto")

# # draw sphere
# u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
# x = np.cos(u)*np.sin(v)
# y = np.sin(u)*np.sin(v)
# z = np.cos(v)
# ax.plot_wireframe(x, y, z, color="r")

In [22]:
batches = [s1d1_array, s1d2_array, s2d1_array, s2d4_array, s3d6_array]

In [23]:
N = 1000
x, y, z = np.random.normal(0, 1, (3, N))

fig = ipv.figure()
scatter = ipv.scatter(x, y, z)
ipv.show()

In [24]:
x.shape

(1000,)

In [25]:
s1d1_mean = np.mean(s1d1_array, axis=0)+5
s1d1_std = np.std(s1d1_array, axis=0)

rvals = 2*np.random.rand(s1d1_mean.shape[0],1)-1
elevation = np.arcsin(rvals)

azimuth = 2*np.pi*np.random.rand(s1d1_mean.shape[0],1)
radii = np.ones((s1d1_mean.shape[0], 1))
radii = s1d1_mean
radii = np.reshape(radii, (radii.shape[0], 1))
radii.shape

# [x,y,z] = sph2cart(azimuth,elevation,radii);

z = radii*np.sin(elevation)
x = radii*np.cos(elevation)*np.cos(azimuth)
y = radii*np.cos(elevation)*np.sin(azimuth)

x = np.squeeze(x)
y = np.squeeze(y)
z = np.squeeze(z)
# s = randi([100, 300], dim, 1);
# c = rand(dim, 3);

# %matplotlib inline
# plt.rcParams["figure.figsize"] = [7.00, 8.0]
# plt.rcParams["figure.autolayout"] = True
# fig = plt.figure()
fig = ipv.figure()

scatter = ipv.scatter(x, y, z)

# ax.set_box_aspect((np.ptp(x), np.ptp(y), np.ptp(z)))
# ax.scatter3D(x, y, z, c=s1d1_std)
# x = x.tolist()
# y = y.tolist()
# z = z.tolist()
# x = [ele[0] for ele in x]
# y = [ele[0] for ele in y]
# z = [ele[0] for ele in z]
# for i in range(len(x)):
#     distance = [(x[i]-x[j])**2+(y[i]-y[j])**2+(z[i]-z[j])**2 for j in range(len(x))]
#     top_4 = (heapq.nsmallest(4, distance))
#     nearest_index = [distance.index(top_4[1]), distance.index(top_4[2]), distance.index(top_4[3])]
#     ax.plot([x[i], x[nearest_index[0]]], [y[i], y[nearest_index[0]]], [z[i], z[nearest_index[0]]], color='blue')
#     ax.plot([x[i], x[nearest_index[1]]], [y[i], y[nearest_index[1]]], [z[i], z[nearest_index[1]]], color='blue')
#     ax.plot([x[i], x[nearest_index[2]]], [y[i], y[nearest_index[2]]], [z[i], z[nearest_index[2]]], color='blue')
ipv.show()

In [26]:
x.shape
x = np.squeeze(x)
x.shape

(116490,)

In [27]:
s1d1_mean = np.mean(s1d1_array, axis=0)+5
s1d1_std = np.std(s1d1_array, axis=0)

rvals = 2*np.random.rand(s1d1_mean.shape[0],1)-1
elevation = np.arcsin(rvals)

azimuth = 2*np.pi*np.random.rand(s1d1_mean.shape[0],1)
radii = np.ones((s1d1_mean.shape[0], 1))
radii = s1d1_mean
radii = np.reshape(radii, (radii.shape[0], 1))
radii.shape

# [x,y,z] = sph2cart(azimuth,elevation,radii);

z = radii*np.sin(elevation)
x = radii*np.cos(elevation)*np.cos(azimuth)
y = radii*np.cos(elevation)*np.sin(azimuth)

# s = randi([100, 300], dim, 1);
# c = rand(dim, 3);

# %matplotlib inline
# plt.rcParams["figure.figsize"] = [7.00, 8.0]
# plt.rcParams["figure.autolayout"] = True
# fig = plt.figure()
fig = ipv.figure()
ax = fig.add_subplot(projection='3d')
ax.set_box_aspect((np.ptp(x), np.ptp(y), np.ptp(z)))
ax.scatter3D(x, y, z, c=s1d1_std)
x = x.tolist()
y = y.tolist()
z = z.tolist()
x = [ele[0] for ele in x]
y = [ele[0] for ele in y]
z = [ele[0] for ele in z]
for i in range(len(x)):
    distance = [(x[i]-x[j])**2+(y[i]-y[j])**2+(z[i]-z[j])**2 for j in range(len(x))]
    top_4 = (heapq.nsmallest(4, distance))
    nearest_index = [distance.index(top_4[1]), distance.index(top_4[2]), distance.index(top_4[3])]
    ax.plot([x[i], x[nearest_index[0]]], [y[i], y[nearest_index[0]]], [z[i], z[nearest_index[0]]], color='blue')
    ax.plot([x[i], x[nearest_index[1]]], [y[i], y[nearest_index[1]]], [z[i], z[nearest_index[1]]], color='blue')
    ax.plot([x[i], x[nearest_index[2]]], [y[i], y[nearest_index[2]]], [z[i], z[nearest_index[2]]], color='blue')
ipv.show()

AttributeError: 'Figure' object has no attribute 'add_subplot'

In [ ]:
input_ = ad.read_h5ad(par['input_train_mod2'])
s1d1 = input_[input_.obs["batch"] == "s1d1", :]
RNA_s1d1 = s1d1
s1d1_array = RNA_s1d1.X.toarray()
s1d1_mean = np.mean(s1d1_array, axis=0)+10
s1d1_std = np.std(s1d1_array, axis=0)

rvals = 2*np.random.rand(s1d1_mean.shape[0],1)-1
elevation = np.arcsin(rvals)

azimuth = 2*np.pi*np.random.rand(s1d1_mean.shape[0],1)
radii = np.ones((s1d1_mean.shape[0], 1))
radii = s1d1_mean
radii = np.reshape(radii, (radii.shape[0], 1))
radii.shape

# [x,y,z] = sph2cart(azimuth,elevation,radii);

z = radii*np.sin(elevation)
x = radii*np.cos(elevation)*np.cos(azimuth)
y = radii*np.cos(elevation)*np.sin(azimuth)

# s = randi([100, 300], dim, 1);
# c = rand(dim, 3);

# %matplotlib inline
plt.rcParams["figure.figsize"] = [7.00, 8.0]
plt.rcParams["figure.autolayout"] = True
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
# ax.axis('equal')
# ax.set_aspect('equal')
ax.set_box_aspect((np.ptp(x), np.ptp(y), np.ptp(z)))
ax.scatter3D(x, y, z, c=s1d1_std, s = s1d1_std);
x = x.tolist()
y = y.tolist()
z = z.tolist()
x = [ele[0] for ele in x]
y = [ele[0] for ele in y]
z = [ele[0] for ele in z]
for i in range(len(x)):
    distance = [(x[i]-x[j])**2+(y[i]-y[j])**2+(z[i]-z[j])**2 for j in range(len(x))]
    top_4 = (heapq.nsmallest(4, distance))
    nearest_index = [distance.index(top_4[1]), distance.index(top_4[2]), distance.index(top_4[3])]
    ax.plot([x[i], x[nearest_index[0]]], [y[i], y[nearest_index[0]]], [z[i], z[nearest_index[0]]], color='blue', linewidth=0.5)
    ax.plot([x[i], x[nearest_index[1]]], [y[i], y[nearest_index[1]]], [z[i], z[nearest_index[1]]], color='blue', linewidth=0.5)
    ax.plot([x[i], x[nearest_index[2]]], [y[i], y[nearest_index[2]]], [z[i], z[nearest_index[2]]], color='blue', linewidth=0.5)

In [ ]:
# a = matlab.double(s1d1_mean)
# b = matlab.double(s1d1_std)

In [ ]:
# ret = eng.sphere_creation(a, b)

In [ ]:
outputs = np.concatenate((batches), axis=0)

In [ ]:
outputs_torch = torch.from_numpy(outputs)
outputs_torch = outputs_torch.float()
# outputs_gpu = outputs_torch.to(device)

In [ ]:
for i in range(5):
    one_hot_encoder = np.zeros((batches[i].shape[0], 10))
    one_hot_encoder[:, i] = 1
    batches[i] = np.concatenate((batches[i], one_hot_encoder), axis=1)

In [ ]:
eng.sphere_creation(nargout=0)inputs = np.concatenate((batches), axis=0)

In [ ]:
inputs_torch = torch.from_numpy(inputs)
inputs_torch = inputs_torch.float()
# inputs_gpu = inputs_torch.to(device)

In [ ]:
# train_inputs = inputs_gpu
# reain_target = inputs_gpu
# val_inputs = inputs_gpu
# val_targets = inputs_gpu
# test_inputs = inputs_gpu
# test_targets = inputs_gpu

In [ ]:
# s1d1_torch = torch.from_numpy(s1d1_array)
# s1d2_torch = torch.from_numpy(s1d2_array)
# s2d1_torch = torch.from_numpy(s2d1_array)
# s2d4_torch = torch.from_numpy(s2d4_array)
# s3d6_torch = torch.from_numpy(s3d6_array)

# s1d1_torch = s1d1_torch.float()
# s1d2_torch = s1d2_torch.float()
# s2d1_torch = s2d1_torch.float()
# s2d4_torch = s2d4_torch.float()
# s3d6_torch = s3d6_torch.float()

In [ ]:
# s1d1_gpu = s1d1_torch.to(device)
# s1d2_gpu = s1d2_torch.to(device)
# s2d1_gpu = s2d1_torch.to(device)
# s2d4_gpu = s2d4_torch.to(device)
# s3d6_gpu = s3d6_torch.to(device)

In [ ]:
input_feature = inputs.shape[1]
output_feature = outputs.shape[1]

In [ ]:
num_epochs = 500
learning_rate = 0.01
latent_dim = 50
loss_fn = F.mse_loss
batch_size = 8

In [ ]:
train_ds = TensorDataset(inputs_torch, outputs_torch)
train_dl = DataLoader(train_ds, batch_size= 10, shuffle=True)

In [ ]:
# for batch_idx, samples in enumerate(train_dl):
#       print(batch_idx, len(samples))

In [ ]:
class Autoencoder_model(nn.Module):
    def __init__(self):
        super(Autoencoder_model, self).__init__()
        # encoding
        self.encoder = nn.Sequential(
            nn.Linear(input_feature, input_feature//16),
            nn.Dropout(0.25),
            nn.ReLU(),

            nn.Linear(input_feature//16, input_feature//16),
            nn.ReLU(),
        
            nn.Linear(input_feature//16, latent_dim),
            nn.ReLU(),
        )
        # decoding
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim+10, input_feature//16),
            nn.Dropout(0.25),
            nn.ReLU(),

            nn.Linear(input_feature//16, input_feature//16),
            nn.ReLU(),

            nn.Linear(input_feature//16, output_feature),
            # nn.ReLU()
        )

    def forward(self, x):
        latent = self.encoder(x)
        new_latent = np.concatenate(latent, x[:, -10:], axis=1)
        output = self.decoder(latent)
        # print('output type:', output.dtype)
        return latent, output.float()

In [ ]:
def fit(num_epochs, model, loss_fn):
    val_min_loss = float('inf')
    counter = 0
    # print('Model training...')
    for epoch in range(num_epochs):
        for x,y in train_dl:
            # Generate predictions
            model = model.train()
            latent, pred = model(x)

            loss = loss_fn(pred, y)
            loss = loss.float()
            loss.backward()

            opt.step()
            opt.zero_grad()

        if epoch % 100 == 0:
            loss = loss.cpu().detach().numpy()
            model = model.eval()
            
            val_pred = model(val_inputs)
            val_loss = loss_fn(val_pred, val_targets)
            val_loss = torch.sqrt(val_loss)
            val_loss = val_loss.cpu().detach().numpy()
            
            train_pred = model(train_inputs)
            train_loss = loss_fn(train_pred, train_targets)
            train_loss = torch.sqrt(train_loss)
            train_loss = train_loss.cpu().detach().numpy()
            
            test_pred = model(test_inputs)
            test_loss = loss_fn(test_pred, test_targets)
            test_loss = torch.sqrt(test_loss)
            test_loss = test_loss.cpu().detach().numpy()

            print('Epoch ', epoch, 'Train_loss: ', train_loss, ' Validation_loss: ', val_loss, ' Test_loss: ', test_loss)
    # train_pred = train_pred.cpu().detach().numpy()
    # val_pred = val_pred.cpu().detach().numpy()
    # test_pred = test_pred.cpu().detach().numpy()
    
    return train_pred, val_pred, test_pred

In [ ]:
model = Autoencoder_modele()
model = model.to(device)
opt = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
train_pred, val_pred, test_pred = fit(num_epochs, model, loss_fn)